<a href="https://colab.research.google.com/github/bmeko/socket_programming/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [64]:
import random 
import json
import numpy as np
import pickle

import nltk
from nltk.stem import WordNetLemmatizer
import tensorflow 
tensorflow.device('/device:GPU:0')
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Activation,Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import load_model

In [56]:
lemmatizer = WordNetLemmatizer()
data = json.load(open("/content/chat-bot/intents.json"))

intents=data["data"][0]
catagory=data["data"][1]

words=[]
conver=[]
documents=[]
ignore_letters = ["?", "!",".", ",", "is", "and", "of"] 

for intent in intents["intents"]:
  for pattern in intent["patterns"]:
    word_list=nltk.word_tokenize(pattern)
    words.extend(word_list)
    documents.append((word_list,intent["tag"]))
    if intent["tag"] not in conver:
      conver.append(intent["tag"])

words=[lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words=sorted(set(words))

conver=sorted(set(conver))

pickle.dump(words,open('/content/chat-bot/words.pkl','wb'))
pickle.dump(conver,open('/content/chat-bot/conver.pkl','wb'))




In [ ]:
from prompt_toolkit.completion import word_completer
training=[]
output_empty= [0]* len(conver)


for document in documents:
  bag=[]
  word_patterns= document[0]
  word_patterns= [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
  for word in words:
    bag.append(1) if word in word_patterns else bag.append(0)
  
  output_row = list(output_empty)
  output_row[conver.index(document[1])]=1
  training.append([bag,output_row])
  

random.shuffle(training)
training=np.array(training)

train_x = list(training[:,0])
train_y = list(training[:,1])

model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),),activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

hist  = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=10, verbose=1)
model.save('conversation_model.model', hist)
print("done")


In [ ]:
words_t=pickle.load(open("/content/chat-bot/words.pkl",'rb'))
conver_t=pickle.load(open("/content/chat-bot/conver.pkl",'rb'))
model= load_model('conversation_model.model')
def clean_sent(sent):
  sent_words=nltk.word_tokenize(sent)
  sent_words=[lemmatizer.lemmatize(word) for word in sent_words]
  return sent_words

def bag_of_word(sent):
  sent_words= clean_sent(sent)
  bag = [0]*len(words_t)
  for w in sent_words:
    for i, word in enumerate(words):
      if word == w:
        bag[i] = 1
  return np.array(bag)

def predict_conv(sent):
  bow = bag_of_word(sent)
  res= model.predict(np.array([bow]))[0]
  error_thresh=0.25
  results =[[i,r] for i, r  in enumerate(res) if r> error_thresh]  
  
  results.sort(key=lambda x:x[1], reverse=True)
  return_list = []
  print(results)
  for r in results:
    return_list.append({'intent': conver_t[r[0]],'probability': str(r[1])})
  print(return_list)

while True:
  message = input(">>")
  ints= predict_conv(message)